In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm

In [2]:
data = pd.read_csv('usdjpy.csv', decimal='.')
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['Rendimientos'] = np.log(data['USD por 1 JPY'] / data['USD por 1 JPY'].shift(1))


In [69]:
def backtesting_delta_normal(data, monto, nc, posicion, n_muestras):

    medias, volatilidades, VaR, CVaR, fechas = [], [], [], [], []
    perdidas = []
    excedentes = []

    monto = float(monto)
    nc = float(nc)
    posicion = posicion.lower()
    n_muestras = int(n_muestras)

    if 'Date' not in data.columns:
        print('No hay datos. Vuelve a intentar con DF de columnas "Date" en orden ascendente y el precio del activo solamente.')
        return

    if monto < 0 or monto > 100000000:
        print("Monto fuera de rango (debe estar entre 0 y 100,000,000). Vuelve a intentar")
        return

    if nc < 0.01 or nc > 0.99:
        print("Nivel de confianza fuera de rango (debe estar entre 0.01 y 0.99). Vuelve a intentar")
        return

    if posicion not in ["long", "larga", "l", "short", "corta", "s", "c"]:
        print('Posición incorrecta. Vuelve a intentar')
        return

    if n_muestras < 0 or n_muestras > len(data)/2 - 1:
        print("Número de muestras inválido, debes ingresar un número positivo menor a la mitad de datos totales")
        return

    z = norm.ppf(1 - nc)
    fi = norm.pdf(z)

    muestras_df = pd.DataFrame()
    muestras_df = data.tail(n_muestras*2)

    for i in range(n_muestras):
        rango = data.iloc[1+i:-(n_muestras-i)]
        medias.append(rango.Rendimientos.mean())
        volatilidades.append(rango.Rendimientos.std(ddof=1))
        VaR.append(abs((medias[i] + z * volatilidades[i]) * monto))
        CVaR.append((medias[i] + (fi / (1-nc)*volatilidades[i]))*monto)
        perdidas.append(monto * max(data.Rendimientos.iloc[-(n_muestras-i)],0))
        excedentes.append(int(CVaR[i] < perdidas[i]))



    results_df = pd.DataFrame(index=range(n_muestras))
    results_df['medias'] = medias
    results_df['volatilidades'] = volatilidades
    results_df['VaRs'] = VaR
    results_df['CVaR'] = CVaR
    results_df['perdida?'] = perdidas
    results_df['excedente (falla)'] = excedentes

    return  results_df


In [71]:
resultados = backtesting_delta_normal(data, 50000, 0.90, 'short', 60)
resultados

,medias,volatilidades,VaRs,CVaR,perdida?,excedente (falla)
0,-0.000388,0.005962,401.459355,503.745795,3.609071,0
1,-0.000388,0.005962,401.450229,503.756745,276.907095,0
2,-0.000384,0.005965,401.405398,504.220042,0.000000,0
3,-0.000407,0.005992,404.274204,505.400982,370.741662,0
4,-0.000396,0.005996,404.015469,506.322386,90.164643,0
5,-0.000387,0.005992,403.318019,506.457429,0.000000,0
6,-0.000388,0.005993,403.388346,506.451474,0.000000,0
7,-0.000395,0.005997,404.031296,506.471371,0.000000,0
8,-0.000404,0.005997,404.487830,506.077577,0.000000,0
9,-0.000410,0.006001,404.987820,506.070669,579.828151,1


In [78]:
total_excedentes = resultados['excedente (falla)'].sum()
pct_excedentes = total_excedentes / len(resultados) * 100
print("Total de excedentes: " + str(total_excedentes),"\n" + "% Total de excedentes: " + str(pct_excedentes)+"%")

Total de excedentes: 3 
% Total de excedentes: 5.0%
